### Librerias necesarias 

In [ ]:
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import pandas as pd
import numpy as np
import os
import cv2
import keras
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn import svm

: 

### Lectura y preprocesamiento de imagenes para el entrenamiento del modelo 

In [ ]:
images1 = "./Original/Benign"
images2 = "./Original/Early"
images3 = "./Original/Pre"
images4 = "./Original/Pro"

#Tamaño de la imagen requerido para este modelo
images_size = 224

: 

### Diccionario de clasificacion de imagenes por tipo 

In [ ]:
codigo = {"Benign": 0, "Early": 1, "Pre": 2, "Pro": 3}
def getcodigo(n):
    for x, y in code.items():
        if n == y:
            return x

: 

In [ ]:
x_train = []
y_train = []
files_dir = os.listdir(images1)
for file_dir in files_dir:
    image = images1 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_train.append(list(image_resized))
    y_train.append(codigo["Benign"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(images2)
for file_dir in files_dir:
    image = images2 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_train.append(list(image_resized))
    y_train.append(codigo["Early"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(images3)
for file_dir in files_dir:
    image = images3 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_train.append(list(image_resized))
    y_train.append(codigo["Pre"])
   
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(images4)
for file_dir in files_dir:
    image = images4 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_train.append(list(image_resized))
    y_train.append(codigo["Pro"])
    
print("Imagenes procesadas correctamente...")

: 

In [ ]:
len(x_train)

: 

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_train),36))) :
    plt.subplot(6,6,n+1)
    plt.imshow(x_train[i])
    plt.axis('off')
    plt.title(y_train[i])

: 

In [ ]:
segmented1 = "./Segmented/Benign"
segmented2 = "./Segmented/Early"
segmented3 = "./Segmented/Pre"
segmented4 = "./Segmented/Pro"

: 

In [ ]:
x_test = []
y_test = []
files_dir = os.listdir(segmented1)
for file_dir in files_dir:
    image = segmented1 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_test.append(list(image_resized))
    y_test.append(codigo["Benign"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(segmented2)
for file_dir in files_dir:
    image = segmented2 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_test.append(list(image_resized))
    y_test.append(codigo["Early"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(segmented3)
for file_dir in files_dir:
    image = segmented3 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_test.append(list(image_resized))
    y_test.append(codigo["Pre"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
files_dir = os.listdir(segmented4)
for file_dir in files_dir:
    image = segmented4 + "/" + file_dir
    image_readed = cv2.imread(image)
    image_resized = cv2.resize(image_readed, (images_size, images_size))
    x_test.append(list(image_resized))
    y_test.append(codigo["Pro"])
print("Imagenes procesadas correctamente...")

: 

In [ ]:
len(x_test)

: 

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_test),36))) :
    plt.subplot(6,6,n+1)
    plt.imshow(x_test[i])
    plt.axis('off')
    plt.title(y_test[i])

: 

In [ ]:
dataset = []
for i, j in zip(x_train, y_train):
    dataset.append([i,j])

: 

In [ ]:
len(dataset)

: 

### Aqui es donde se pieza a preparar los datos para el entrenamiento del modelo 

In [ ]:
x = []
y = []
for img, titulo in dataset:
    x.append(img)
    y.append(titulo)

: 

In [ ]:
#Convertimos a array la lista de valores
x = np.array(x)
y = np.array(y)

: 

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size = 0.8, shuffle = True)
print(xtest.shape)
print(xtrain.shape)

: 

In [ ]:
modelo_base = tf.keras.applications.vgg19.VGG19()
modelo = tf.keras.models.Sequential()

for layer in modelo_base.layers[:-1]:
    modelo.add(layer)

for layer in modelo.layers:
    layer.trainable = False
    
modelo.add(tf.keras.layers.Dense(4, activation = tf.nn.softmax))

: 

In [ ]:
modelo.summary()

: 

### Entrenamiento del modelo 

In [ ]:
modelo.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics=['accuracy'])

: 

In [ ]:
historial = modelo.fit(xtrain,
                       ytrain,
                       batch_size=128,
                       verbose=1,
                       validation_data=(xtest,ytest),
                       epochs=20) 

: 

In [ ]:
y_pred = modelo.predict(xtest)

print("Predicción: {}".format(y_pred.shape))

: 

In [ ]:
plt.plot(historial.history['accuracy'])
plt.plot(historial.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['train', 'test'], loc = 'lower right')
plt.tight_layout()

: 

### Guardar el modelo 

In [ ]:
modelo.save('vgg19_leukemia_detection.h5')
print("Modelo guardado...")

: 

### Sección de clasificacion de imagenes 

In [ ]:
# Cargar el modelo entrenado desde el archivo .h5
modelo_cargado = tf.keras.models.load_model('vgg19_leukemia_detection.h5')  # Reemplaza con la ruta correcta

# Tamaño de entrada esperado por el modelo
input_size = (224, 224)

def cargar_y_preprocesar_imagen(ruta_imagen):
    # Cargar la imagen
    img = image.load_img(ruta_imagen, target_size=input_size)
    img_array = image.img_to_array(img)

    # Agregar una dimensión para el lote (batch)
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocesar la imagen
    img_array = tf.keras.applications.vgg19.preprocess_input(img_array)

    return img_array

def clasificar_imagen(ruta_imagen):
    # Cargar y preprocesar la imagen
    img_array = cargar_y_preprocesar_imagen(ruta_imagen)

    # Realizar la predicción
    prediction = modelo_cargado.predict(img_array)

    # Obtener la clase predicha (0 si es Benign, 1 si es Early, 2 si es Pre, 3 si es Pro)
    predicted_class = np.argmax(prediction)

    # Mapear la clase predicha a una etiqueta de texto utilizando el diccionario code
    code = {0: "Benign", 1: "Early", 2: "Pre", 3: "Pro"}
    predicted_class_name = code[predicted_class]

    return predicted_class_name

# Ejemplo de clasificación de una imagen individual
imagen_prueba = './all1.bmp'  # Reemplaza con la ruta real de tu imagen
resultado = clasificar_imagen(imagen_prueba)
print(f'La imagen se clasifica como: {resultado}')

: 

In [ ]:
grid_svm = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma':[1, 0.1, 0.01, 0.001, 0.0001 ],
    'kernel': ['rbf', 'poly']
}

SVM_grid = GridSearchCV(svm.SVC(), grid_svm, cv=5)
SVM_grid.fit(xtrain, ytrain)

: 

: 